In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

In [53]:
cd '/content/drive/My Drive/Colab Notebooks/dw-matrix/matrix2/dw-matrix-car'

/content/drive/My Drive/Colab Notebooks/dw-matrix/matrix2/dw-matrix-car


In [54]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df [feat][0],list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [57]:
cat_feats = [x for  x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for  x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model, feats):
  x = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, x, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

[22:31:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:32:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


-13108.379065811214

In [76]:
df['param_pojemność-skokowa']=df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if  str(x) == 'None' else int(str(x).split(' ')[0]) )
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if  str(x) == 'None' else int (x))
feats = ['param_napęd__cat', 'param_faktura-vat__cat', 'param_stan__cat', 'param_rok-produkcji', 'param_moc', 'feature_kamera-cofania__cat', 'param_skrzynia-biegów__cat', 'param_marka-pojazdu__cat', 'param_pojemność-skokowa', 'feature_bluetooth__cat', 'feature_łopatki-zmiany-biegów__cat', 'feature_światła-led__cat', 'feature_klimatyzacja-manualna__cat', 'param_kod-silnika__cat', 'feature_gniazdo-aux__cat', 'feature_klimatyzacja-automatyczna__cat', 'feature_radio-fabryczne__cat', 'feature_czujniki-parkowania-tylne__cat', 'feature_poduszki-boczne-tylne__cat', 'feature_odtwarzacz-dvd__cat',]

xgb_params = {
  'max_depth': 5,
  'n_estimators': 50,
  'learning_rate': 0.1,
  'seed': 0
}
run_model(xgb.XGBRegressor(**xgb_params), feats)

[23:47:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:47:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:47:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10000.392884894685, 82.59828921222274)

In [77]:
def obj_func(params):
  print("Training with params: ")
  print(params)
  mean_mae, score_std = run_model(xgb.XGBRegressor(**params),feats)
  return{'loss': np.abs(mean_mae), 'status': STATUS_OK}
# space
xgb_reg_params = {
'learning rate': hp.choice('learning rate', np.arange(0.05, 0.31, 0.05)),
'max_depth': hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
'objective': 'reg:squarederror',
'n_estimators': 100,
'seed': 0,
}
## run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Training with params: 
{'colsample_bytree': 0.75, 'learning rate': 0.05, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8500000000000001}
Training with params: 
{'colsample_bytree': 0.55, 'learning rate': 0.05, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.6000000000000001}
Training with params: 
{'colsample_bytree': 0.55, 'learning rate': 0.25, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.65, 'learning rate': 0.3, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.9, 'learning rate': 0.2, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning rate': 0.3, 'max_dept

{'colsample_bytree': 0.8500000000000001,
 'learning rate': 4,
 'max_depth': 8,
 'subsample': 0.9}